In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [6]:
sector_zscores=sector_XS_normalized(amihud)

# Annual portfolio sorts

In [7]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual_inverse_vol(sector_zscores[s],cleansed,3)

tsmom_model.py:249: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  vol=pd.DataFrame.rolling(price_data.resample(rule='m',how='last').pct_change(),36).std().replace(0,1)
tsmom_model.py:257: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
tsmom_model.py:261: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = w.multiply(price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)]).sum(axis=1)


In [8]:
for sec in sector_portfolio_sorts.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())


Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.66 &   0.60 &   0.99 &    0.33 \\
Standard Deviation           &  15.88 &  12.05 &  15.36 &   14.69 \\
Information Ratio            &   0.50 &   0.60 &   0.77 &    0.27 \\
Skewness                     &  -0.19 &  -0.55 &  -0.09 &    0.17 \\
Excess Kurtosis              &   0.72 &   2.86 &   0.34 &   -0.02 \\
CAPM Alpha Annualized (in \%) &   4.24 &   4.54 &   8.60 &    4.36 \\
CAPM Alpha Tstat             &   2.36 &   2.84 &   4.33 &    1.36 \\
CAPM Beta (in \%)             &   1.02 &   0.74 &   0.90 &   -0.12 \\
CAPM Beta Tstat              &  19.83 &  16.48 &  21.37 &   -1.60 \\
\$R\textasciicircum2\$                        &   0.76 &   0.69 &   0.63 &    0.01 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &     1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.17 &  0.39 &   0.42 &    0.26 \\
Standard Deviation     

# FHT Annual portfolio sorts

In [9]:
FHT=calculate_FHT(cleansed)
sector_zscores=sector_XS_normalized(FHT)

tsmom_model.py:360: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:361: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:362: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:362: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [10]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual_inverse_vol(sector_zscores[s].dropna(how='all'),cleansed,3)

In [11]:
for sec in sector_portfolio_sorts_FHT.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts_FHT[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.37 &   0.53 &   0.73 &    0.36 \\
Standard Deviation           &  12.34 &  15.75 &  16.35 &   17.00 \\
Information Ratio            &   0.36 &   0.40 &   0.53 &    0.25 \\
Skewness                     &  -0.10 &  -0.09 &   0.08 &    0.27 \\
Excess Kurtosis              &   0.86 &   1.37 &   1.31 &    0.70 \\
CAPM Alpha Annualized (in \%) &   4.66 &   6.28 &   5.52 &    0.86 \\
CAPM Alpha Tstat             &   2.55 &   3.43 &   2.47 &    0.24 \\
CAPM Beta (in \%)             &   0.71 &   1.04 &   1.02 &    0.31 \\
CAPM Beta Tstat              &  14.89 &  20.66 &  19.20 &    3.65 \\
\$R\textasciicircum2\$                        &   0.58 &   0.77 &   0.70 &    0.06 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &     0 &      1 &     2 &  Factor \\
\midrule
Monthly Return (in \%)        &  0.14 &   0.36 &  0.31 &    0.17 \\
Standard Deviation       

# Monthly portoflio sorts

In [40]:
amihud_monthly = read_monthly(amihud=True,sorts=3,xs=True,IV=True)

In [41]:
for sec in amihud_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(amihud_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(amihud_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.55 &   0.77 &   1.01 &    0.47 \\
Standard Deviation           &  15.79 &  13.16 &  17.52 &   15.64 \\
Information Ratio            &   0.42 &   0.70 &   0.69 &    0.36 \\
Skewness                     &  -0.07 &  -0.57 &   0.48 &    0.45 \\
Excess Kurtosis              &   0.81 &   2.65 &   3.93 &    2.16 \\
CAPM Alpha Annualized (in \%) &   3.49 &   6.82 &   8.89 &    5.41 \\
CAPM Alpha Tstat             &   1.84 &   3.31 &   3.78 &    1.52 \\
CAPM Beta (in \%)             &   0.98 &   0.77 &   1.03 &    0.06 \\
CAPM Beta Tstat              &  18.05 &  14.04 &  16.31 &    0.62 \\
\$R\textasciicircum2\$                        &   0.73 &   0.65 &   0.66 &    0.00 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &     1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.19 &  0.27 &   0.40 &    0.21 \\
Standard Deviation     

# Monthly FHT

In [44]:
FHT_monthly = read_monthly(amihud=False,sorts=3,xs=True,IV=True)

In [45]:
for sec in FHT_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(FHT_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(FHT_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())



Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.87 &   0.35 &   0.40 &   -0.47 \\
Standard Deviation           &  11.86 &  15.83 &  18.06 &   17.46 \\
Information Ratio            &   0.88 &   0.27 &   0.26 &   -0.33 \\
Skewness                     &   0.14 &  -0.16 &   0.25 &    0.21 \\
Excess Kurtosis              &   0.96 &   0.62 &   1.74 &    1.36 \\
CAPM Alpha Annualized (in \%) &   9.59 &   5.29 &   1.71 &   -7.88 \\
CAPM Alpha Tstat             &   4.50 &   2.61 &   0.82 &   -2.19 \\
CAPM Beta (in \%)             &   0.66 &   1.06 &   1.17 &    0.51 \\
CAPM Beta Tstat              &  13.06 &  21.66 &  21.69 &    5.71 \\
\$R\textasciicircum2\$                        &   0.51 &   0.72 &   0.75 &    0.17 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.15 &   0.28 &   0.33 &    0.18 \\
Standard Deviation   

In [30]:
# Spanning Test
